In [1]:
import hail as hl

Loading BokehJS ...

In [2]:
hl.init()

Py4JJavaError: An error occurred while calling z:is.hail.backend.spark.SparkBackend.apply.
: java.io.FileNotFoundException: File file:/tmp/spark-events does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:779)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1100)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:769)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:462)
	at org.apache.spark.deploy.history.EventLogFileWriter.requireLogBaseDirAsDirectory(EventLogFileWriters.scala:77)
	at org.apache.spark.deploy.history.SingleEventLogFileWriter.start(EventLogFileWriters.scala:221)
	at org.apache.spark.scheduler.EventLoggingListener.start(EventLoggingListener.scala:81)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:632)
	at is.hail.backend.spark.SparkBackend$.configureAndCreateSparkContext(SparkBackend.scala:170)
	at i

In [3]:
! pip install pysam

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 136.1 kB/s eta 0:00:00m eta 0:00:010:00:06


In [4]:
import os
import random
import pandas as pd
import pysam

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [5]:
VCF_DIR = '/home/julia/Documents/thesis/vcf_for_generate/'

In [6]:
# Загрузка таблицы с полами
sex_info = {
    "000007000020": "ж",
    "000007000030": "м",
    "000007000040": "м",
    "000007000060": "м",
    "000007000070": "м"
}

In [7]:
old_ids = list(sex_info.keys())
print(old_ids)

['000007000020', '000007000030', '000007000040', '000007000060', '000007000070']


In [8]:
new_ids = [f"{70000080 + i:012d}" for i in range(10)]  # 000007000080 ... 000007000179

In [9]:
print(new_ids)

['000070000080', '000070000081', '000070000082', '000070000083', '000070000084', '000070000085', '000070000086', '000070000087', '000070000088', '000070000089']


In [ ]:
# Создаем 10 новых файлов
for new_id in new_ids:
    # 1. Выбираем случайный исходный образец
    original_id = random.choice(old_ids)
    original_file = f"{VCF_DIR + original_id}.vcf.gz"
    sex = sex_info[original_id]  # пол исходного образца
    
        # 2. Читаем исходный VCF (распаковываем на лету)
    with pysam.VariantFile(original_file) as vcf:
        # 3. Создаем новый VCF (несжатый)
        output_vcf = f"{VCF_DIR + new_id}.vcf"
        with pysam.VariantFile(output_vcf, "w", header=vcf.header) as out:
            for record in vcf:
                if random.random() > 0.2:  # Удаляем 20% вариантов
                    if record.chrom == "chrX" and sex == "м":
                        if random.random() > 0.5:
                            out.write(record)
                    else:
                        out.write(record)
            
            # 4. Добавляем новые варианты (10% от исходного количества)
            for _ in range(int(0.1 * len(list(vcf.fetch())))):
                chrom = random.choice(["chr1", "chr2", "chr3", "chrX"])
                pos = random.randint(1_000_000, 100_000_000)
                ref = random.choice(["A", "C", "G", "T"])
                alt = random.choice(["A", "C", "G", "T"])
                if ref != alt:
                    # record = out.new_record(
                    #     contig=chrom, start=pos, stop=pos, ref=ref, alts=[alt],
                    #     qual=30, filter="PASS"
                    # )
                    record = out.new_record(
                        contig=chrom,
                        start=pos,  # POS в VCF (1-based)
                        stop=pos,    # Для однонуклеотидных вариантов = POS
                        ref=ref,
                        alts=[alt],
                        qual=30,     # QUAL (может быть 30, 100 или другое значение)
                        filter="PASS",
                        info=None      # Обязательное поле INFO (может быть пустым)
                    )
                    out.write(record)

# 5. Обновляем таблицу полов
new_sex_info = {
    "ID": new_ids,
    "sex": [random.choices(["м", "ж"], weights=[0.7, 0.3])[0] for _ in range(10)],
    "profile": ["WGS"] * 10
}
# pd.DataFrame(new_sex_info).to_csv(VCF_DIR + "/extended_sex_info.csv", index=False)

In [14]:
def generate_synthetic_vcf(original_file, output_file, deletion_rate=0.2, addition_rate=0.1):
    original_file = VCF_DIR + original_file + '.vcf.gz'
    with pysam.VariantFile(original_file, 'r') as vcf_in:
        # Копируем оригинальный заголовок
        header = vcf_in.header.copy()
        
        with pysam.VariantFile(output_file, 'w', header=header) as vcf_out:
            # Список для хранения оригинальных вариантов
            original_variants = list(vcf_in.fetch())
            
            # Фильтрация и сохранение вариантов
            for record in original_variants:
                # Пропускаем варианты с вероятностью deletion_rate
                if random.random() > deletion_rate:
                    vcf_out.write(record)
            
            # Добавление новых случайных вариантов
            new_variants_count = int(len(original_variants) * addition_rate)
            for _ in range(new_variants_count):
                # Создание нового варианта
                new_record = vcf_out.new_record()
                
                # Случайный выбор хромосомы
                chromosomes = list(header.contigs.keys())
                new_record.chrom = random.choice(chromosomes)
                
                # Случайная позиция
                new_record.pos = random.randint(1, header.contigs[new_record.chrom].length)
                
                # Случайные ref и alt аллели
                bases = ['A', 'C', 'G', 'T']
                new_record.ref = random.choice(bases)
                alt_bases = [base for base in bases if base != new_record.ref]
                new_record.alts = (random.choice(alt_bases),)
                
                # Установка качества
                new_record.qual = random.uniform(10, 100)
                
                # Добавление фильтра
                new_record.filter.add('PASS')
                
                vcf_out.write(new_record)

# Пример использования
def generate_synthetic_vcfs(input_files, output_dir, new_ids):
    for new_id, input_file in zip(new_ids, input_files):
        output_file = f"{output_dir}/{new_id}.vcf"
        generate_synthetic_vcf(input_file, output_file)

# Пример вызова
input_files = old_ids
n_ids = ['000070000085', '000070000086', '000070000087', '000070000088', '000070000089']
generate_synthetic_vcfs(input_files, VCF_DIR, n_ids)

[W::hts_idx_load3] The index file is older than the data file: /home/julia/Documents/thesis/vcf_for_generate/000007000030.vcf.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: /home/julia/Documents/thesis/vcf_for_generate/000007000060.vcf.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: /home/julia/Documents/thesis/vcf_for_generate/000007000070.vcf.gz.tbi


In [13]:
print(new_ids)

['000070000080', '000070000081', '000070000082', '000070000083', '000070000084', '000070000085', '000070000086', '000070000087', '000070000088', '000070000089']


In [ ]:
# 5. Обновляем таблицу полов
new_sex_info = {
    "ID": new_ids,
    "sex": [random.choices(["м", "ж"], weights=[0.7, 0.3])[0] for _ in range(10)],
    "profile": ["WGS"] * 10
}
pd.DataFrame(new_sex_info).to_csv(VCF_DIR + "/extended_sex_info.csv", index=False)

In [15]:
def generate_synthetic_vcf_v2(original_file, output_file, deletion_rate=0.2, addition_rate=0.1):
    original_file = VCF_DIR + original_file + '.vcf.gz'
    with pysam.VariantFile(original_file, 'r') as vcf_in:
        header = vcf_in.header.copy()
        
        # Определяем стандартные хромосомы
        standard_chromosomes = [f'chr{i}' for i in range(1, 23)] + ['chrX', 'chrY']
        
        with pysam.VariantFile(output_file, 'w', header=header) as vcf_out:
            original_variants = list(vcf_in.fetch())
            
            for record in original_variants:
                if random.random() > deletion_rate:
                    # Заменяем имя хромосомы на стандартное
                    chrom = record.chrom.split('_')[0]  # Берем часть до первого _
                    if chrom in standard_chromosomes:
                        record.chrom = chrom
                        vcf_out.write(record)
            
            new_variants_count = int(len(original_variants) * addition_rate)
            for _ in range(new_variants_count):
                new_record = vcf_out.new_record()
                new_record.chrom = random.choice(standard_chromosomes)  # Только стандартные
                new_record.pos = random.randint(1, 100_000_000)  # Фиксированный диапазон
                
                bases = ['A', 'C', 'G', 'T']
                new_record.ref = random.choice(bases)
                new_record.alts = (random.choice([b for b in bases if b != new_record.ref]),)
                
                new_record.qual = random.uniform(10, 100)
                new_record.filter.add('PASS')
                
                vcf_out.write(new_record)

In [18]:
n_ids = ['000070000080', '000070000081', '000070000082', '000070000083', '000070000084', '000070000085', '000070000086', '000070000087', '000070000088', '000070000089', '000070000089']

In [17]:
def generate_synthetic_vcfs2(input_files, output_dir, new_ids):
    for new_id, input_file in zip(new_ids, input_files):
        output_file = f"{output_dir}/{new_id}.vcf"
        generate_synthetic_vcf_v2(input_file, output_file)

input_files = old_ids
generate_synthetic_vcfs2(input_files, VCF_DIR, n_ids)

In [27]:
# 5. Обновляем таблицу полов
new_sex_info = {
    "ID": n_ids,
    "sex": [random.choices(["м", "ж"], weights=[0.7, 0.3])[0] for _ in range(11)],
    "profile": ["WGS"] * 11
}
pd.DataFrame(new_sex_info).to_csv(VCF_DIR + "extended_sex_info.csv", index=False)
print(new_sex_info)

{'ID': ['000070000080', '000070000081', '000070000082', '000070000083', '000070000084', '000070000085', '000070000086', '000070000087', '000070000088', '000070000089', '000070000089'], 'sex': ['ж', 'м', 'м', 'ж', 'м', 'ж', 'м', 'м', 'ж', 'м', 'ж'], 'profile': ['WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS', 'WGS']}


In [22]:
print(len(n_ids))

11
